In [282]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

In [283]:
data = pd.read_csv('data/train.csv')
census = pd.read_csv('data/census_starter.csv')

In [284]:
test_data = pd.read_csv('data/test.csv')

In [285]:
row_id = test_data['row_id']
test_data = test_data.drop('row_id', axis=1)
test_data['first_day_of_month'] = pd.to_datetime(test_data['first_day_of_month'])
test_data['year'] = test_data['first_day_of_month'].dt.year.astype('category') 
test_data['month'] = test_data['first_day_of_month'].dt.month.astype('category')
test_data['cfips'] = test_data['cfips'].astype('category')
test_data.drop('first_day_of_month', axis=1, inplace=True) #drop first_day_of_month column

In [286]:
test_data.head()

,cfips,year,month
0,1001,2022,11
1,1003,2022,11
2,1005,2022,11
3,1007,2022,11
4,1009,2022,11


In [287]:
data['first_day_of_month'] = pd.to_datetime(data['first_day_of_month'])
data['year'] = data['first_day_of_month'].dt.year #extact year
data['month'] = data['first_day_of_month'].dt.month #extract month
data.drop('first_day_of_month', axis=1, inplace=True) #drop first_day_of_month column

In [288]:
data.drop('active', axis=1, inplace=True) #drop active column

In [289]:
data.head()

,row_id,cfips,county,state,microbusiness_density,year,month
0,1001_2019-08-01,1001,Autauga County,Alabama,3.007682,2019,8
1,1001_2019-09-01,1001,Autauga County,Alabama,2.884870,2019,9
2,1001_2019-10-01,1001,Autauga County,Alabama,3.055843,2019,10
3,1001_2019-11-01,1001,Autauga County,Alabama,2.993233,2019,11
4,1001_2019-12-01,1001,Autauga County,Alabama,2.993233,2019,12


In [290]:
census.head()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


We have pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc year wise 

The idea would be to map the each of colums with cfips id with year

In [291]:
# check for missing values
census.isna().sum()

pct_bb_2017              0
pct_bb_2018              0
pct_bb_2019              0
pct_bb_2020              1
pct_bb_2021              1
cfips                    0
pct_college_2017         0
pct_college_2018         0
pct_college_2019         0
pct_college_2020         1
pct_college_2021         1
pct_foreign_born_2017    0
pct_foreign_born_2018    0
pct_foreign_born_2019    0
pct_foreign_born_2020    1
pct_foreign_born_2021    1
pct_it_workers_2017      0
pct_it_workers_2018      1
pct_it_workers_2019      0
pct_it_workers_2020      1
pct_it_workers_2021      1
median_hh_inc_2017       0
median_hh_inc_2018       1
median_hh_inc_2019       0
median_hh_inc_2020       2
median_hh_inc_2021       2
dtype: int64

In [292]:
year = [2017, 2018, 2019, 2020, 2021]

for i in year: 
    census['pct_bb_' + str(i)].fillna(census['median_hh_inc_' + str(i)].median(), inplace=True)# fill all nan values with mean
    census['pct_college_' + str(i)].fillna(census['median_hh_inc_' + str(i)].median(), inplace=True)# fill all nan values with mean
    census['pct_foreign_born_' + str(i)].fillna(census['median_hh_inc_' + str(i)].median(), inplace=True)# fill all nan values with mean
    census['pct_it_workers_' + str(i)].fillna(census['median_hh_inc_' + str(i)].median(), inplace=True)# fill all nan values with mean
    census['median_hh_inc_' + str(i)].fillna(census['median_hh_inc_' + str(i)].median(), inplace=True) # fill all nan values with median


In [293]:
# map values pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc year wise in data if cfips is same 
for i in year:
    data['pct_bb'] = data['cfips'].map(census.set_index('cfips')['pct_bb_' + str(i)])
    data['pct_college'] = data['cfips'].map(census.set_index('cfips')['pct_college_' + str(i)])
    data['pct_foreign_born'] = data['cfips'].map(census.set_index('cfips')['pct_foreign_born_' + str(i)])
    data['pct_it_workers'] = data['cfips'].map(census.set_index('cfips')['pct_it_workers_' + str(i)])
    data['median_hh_inc'] = data['cfips'].map(census.set_index('cfips')['median_hh_inc_' + str(i)])    

In [294]:
data.head(100)

,row_id,cfips,county,state,microbusiness_density,year,month,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc
0,1001_2019-08-01,1001,Autauga County,Alabama,3.007682,2019,8,85.5,16.4,2.1,1.1,62660.0
1,1001_2019-09-01,1001,Autauga County,Alabama,2.884870,2019,9,85.5,16.4,2.1,1.1,62660.0
2,1001_2019-10-01,1001,Autauga County,Alabama,3.055843,2019,10,85.5,16.4,2.1,1.1,62660.0
3,1001_2019-11-01,1001,Autauga County,Alabama,2.993233,2019,11,85.5,16.4,2.1,1.1,62660.0
4,1001_2019-12-01,1001,Autauga County,Alabama,2.993233,2019,12,85.5,16.4,2.1,1.1,62660.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1005_2021-01-01,1005,Barbour County,Alabama,1.111998,2021,1,64.6,6.7,2.6,0.8,36422.0
96,1005_2021-02-01,1005,Barbour County,Alabama,1.072105,2021,2,64.6,6.7,2.6,0.8,36422.0
97,1005_2021-03-01,1005,Barbour County,Alabama,1.057146,2021,3,64.6,6.7,2.6,0.8,36422.0
98,1005_2021-04-01,1005,Barbour County,Alabama,1.102025,2021,4,64.6,6.7,2.6,0.8,36422.0


In [295]:
data['month'] = data['month'].astype('category')
data['year'] = data['year'].astype('category')
data['cfips'] = data['cfips'].astype('category')

In [296]:
data.dtypes

row_id                     object
cfips                    category
county                     object
state                      object
microbusiness_density     float64
year                     category
month                    category
pct_bb                    float64
pct_college               float64
pct_foreign_born          float64
pct_it_workers            float64
median_hh_inc             float64
dtype: object

In [297]:
# split data to X and y
X = data[['cfips','year','month']]
y = data['microbusiness_density']

In [298]:
# split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [299]:
from sklearn.preprocessing import OrdinalEncoder

def ordinal_encoder(df,stage):
    cat_cols = df.select_dtypes(include=['category','object']).columns
    ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    if stage == 'train':
        ord_enc.fit(df[cat_cols])
        joblib.dump(ord_enc, 'cache/ordinal_encoder.pkl')
    else:
        ord_enc = joblib.load('cache/ordinal_encoder.pkl')
    df[cat_cols] = ord_enc.transform(df[cat_cols])
    return df

In [300]:
X_train = ordinal_encoder(X_train,'train')

In [301]:
X_test = ordinal_encoder(X_test,'test')

In [302]:
test_data = ordinal_encoder(test_data,'test')

In [303]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((81917, 3), (40348, 3), (81917,), (40348,))

In [304]:
# from sklearn.preprocessing import StandardScaler

# def standard_scaler(df,stage):
#     num_cols = df.select_dtypes(include=['int64','float64']).columns
#     std_scaler = StandardScaler()
#     if stage == 'train':
#         std_scaler.fit(df[num_cols])
#         joblib.dump(std_scaler, 'cache/standard_scaler.pkl')
#     else:
#         std_scaler = joblib.load('cache/standard_scaler.pkl')
#     df[num_cols] = std_scaler.transform(df[num_cols])
#     return df

In [305]:
test_data.shape

(25080, 3)

In [306]:
# X_train = standard_scaler(X_train,'train')

In [307]:
# X_test = standard_scaler(X_test,'test')

In [308]:
# test_data = ordinal_encoder(test_data,'test')

In [309]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
# report performance
def report_performance(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print('MSE: ', mse)
    print('RMSE: ', rmse)
    print('MAE: ', mae)
    print('R2: ', r2)

In [316]:
# train a linear regression model
from sklearn.linear_model import LinearRegression



lr = LinearRegression()
lr.fit(X_train, y_train)

train_score = lr.score(X_train, y_train)
print('Training score: ', train_score)

lr_pred = lr.predict(X_test)

test_score = lr.score(X_test, y_test)
print('Testing score: ', test_score)


report_performance(y_test, lr_pred)

test_data_pred = lr.predict(test_data)


test_score = lr.score(test_data, test_data_pred)
print('Test data score: ', test_score)


Training score:  0.00042573484526586824
Testing score:  0.0005646025966116186
MSE:  21.86433968106804
RMSE:  4.675931958558426
MAE:  2.4502516916032455
R2:  0.0005646025966116186
Test data score:  1.0


In [325]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

train_score = rf.score(X_train, y_train)
print('Training score: ', train_score)

rf_pred = rf.predict(X_test)

test_score = rf.score(X_test, y_test)

print('Testing score: ', test_score)

report_performance(y_test, rf_pred)

rf_test_data_pred = rf.predict(test_data)


test_score = rf.score(test_data, rf_test_data_pred)
print('Test data score: ', test_score)

Training score:  0.9799996567924294
Testing score:  0.8596950490946853
MSE:  3.069408101313438
RMSE:  1.751972631439612
MAE:  0.11089994031763634
R2:  0.8596950490946853
Test data score:  1.0


In [312]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)

train_score = gb.score(X_train, y_train)
print('Training score: ', train_score)

gb_pred = gb.predict(X_test)
test_score = gb.score(X_test, y_test)
print('Testing score: ', test_score)

report_performance(y_test, gb_pred)

Training score:  0.2901315016536159
Testing score:  0.3286094547393088
MSE:  14.687803712351753
RMSE:  3.8324670530027722
MAE:  2.002658250234798
R2:  0.3286094547393088


In [313]:
#XGBoost
import xgboost as xgb
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100, random_state=42)
xgb.fit(X_train, y_train)

train_score = xgb.score(X_train, y_train)
print('Training score: ', train_score)

xgb_pred = xgb.predict(X_test)
test_score = xgb.score(X_test, y_test)
print('Testing score: ', test_score)

report_performance(y_test, xgb_pred)

# for test data

test_data_pred = xgb.predict(test_data)


test_score = xgb.score(test_data, test_data_pred)
print('Test data score: ', test_score)


Training score:  0.832089775777944
Testing score:  0.7644160192487552
MSE:  5.153798026311477
RMSE:  2.270197794534978
MAE:  1.2124084377705497
R2:  0.7644160192487552
Test data score:  1.0


In [326]:
submission = pd.DataFrame({'row_id': row_id, 'microbusiness_density': rf_test_data_pred})
submission.to_csv('data/submission.csv', index=False)